In [1]:
from tqdm import tqdm
from datetime import date
import bs4, requests, json

In [8]:
home_url = r'http://www.mizzimaburmese.com/news'
base_url = 'http://www.mizzimaburmese.com'

# get original page
soup = bs4.BeautifulSoup(requests.get(home_url).content, 'lxml')

# get main menu links
main_menu_list = soup.select('#block-system-main-menu')
print('# main-menu:', len(main_menu_list))

# get all the links from main menu
main_menu_links_list = [base_url+atag['href'] for atag in main_menu_list[0].ul.find_all('a')]
print('# main menu links:', len(main_menu_links_list))

# main-menu: 1
# main menu links: 57


In [13]:
art_url_list = []
article_list = []
for link in tqdm(main_menu_links_list):
    current_page = bs4.BeautifulSoup(requests.get(link).content, 'lxml')
    articles = current_page.select('.news-category-small-image-title')
    art_url_list.extend([base_url+article_header.a['href'] for article_header in articles])

100%|██████████| 57/57 [01:05<00:00,  1.14s/it]


In [15]:
art_url_list_unq = list(set(art_url_list))
print(len(art_url_list), len(art_url_list_unq))

475 375


In [27]:
N = 50 # we are only taking 50

article_list = []
for art_url in tqdm(art_url_list_unq[:N]):
    article = bs4.BeautifulSoup(requests.get(art_url).content, 'lxml')
    art_title = article.select('.news-details-title')[0].text
    art_body = article.article
    article_list.append({
        'url': art_url,
        'title': art_title,
        'body': '\n'.join(para.text for para in article.article.find_all('p'))
    })

100%|██████████| 50/50 [01:11<00:00,  1.44s/it]


In [29]:
with open('data/mizzima_{}.json'.format(date.today()), 'w', encoding='utf-8') as f:
    json.dump(article_list, f, ensure_ascii=False, indent=4)